In [16]:
import tensorflow as tf
import numpy as np
from config import config

In [11]:
# https://arxiv.org/pdf/1704.07911.pdf
# https://arxiv.org/pdf/1708.03798.pdf
# data: https://github.com/udacity/self-driving-car/tree/master/datasets/CH2
# scripts to convert rosbags to csv: https://github.com/rwightman/udacity-driving-reader

# TODO: 
# 1. Input image need to be changed to YUV (img_yuv = cv2.cvtColor(image, cv2.COLOR_BGR2YUV))
# 2. Cropping to 200 x 66 per paper
# 3. Figure out proper steps for training per epoch.
# 4. Split data into train and validation.
# 5. Incorporate testing framework.
# 6. Incorporate visualization.

In [12]:
csv_header = "index,timestamp,width,height,frame_id,filename,angle,torque,speed,lat,long,alt".split(",")
csv_header[-8:-5]

['frame_id', 'filename', 'angle']

In [4]:
home = config['bag4']

In [14]:
def input_fn(file_path):    
    def decode_csv(line):
        # tf.decode_csv needs a record_default as 2nd parameter
        data = tf.decode_csv(line,  list(np.array([""]*12).reshape(12,1)))[-8:-5]
        img_path = home+data[1]
        img_decoded = tf.to_float(tf.image.decode_image(tf.read_file(img_path)))
        
        # normalize between (-1,1)
        img_decoded = -1.0 + 2.0 * img_decoded / 255.0
        steer_angle = tf.string_to_number(data[2], tf.float32)
        
        # return camera as well to confirm location of camera 
        # e.g left_camera, center_camera, right_camera...we want center_camera
        return {'image':  img_decoded, 'camera': data[0]}, [steer_angle]

    # skip() header row,
    # filter() for center camera,
    # map() transform each line by applying decode_csv()
    dataset = (tf.data.TextLineDataset(file_path) 
        .skip(1) 
        .filter(lambda x: tf.equal(
            tf.string_split(tf.reshape(x,(1,)),',').values[4],
            'center_camera'))
        .map(decode_csv)) 
               
    return dataset
    

In [15]:
file_path=home+"interpolated.csv"
dataset = input_fn(file_path)
dataset = dataset.batch(32) 
batch_generator = dataset.make_one_shot_iterator()
batch = batch_generator.get_next()

In [7]:
with tf.Session() as sess:
    sample = sess.run(batch)

In [8]:
# confirm only center camera data returned.
sample[0]['camera']

array(['center_camera', 'center_camera', 'center_camera', 'center_camera',
       'center_camera', 'center_camera', 'center_camera', 'center_camera',
       'center_camera', 'center_camera', 'center_camera', 'center_camera',
       'center_camera', 'center_camera', 'center_camera', 'center_camera',
       'center_camera', 'center_camera', 'center_camera', 'center_camera',
       'center_camera', 'center_camera', 'center_camera', 'center_camera',
       'center_camera', 'center_camera', 'center_camera', 'center_camera',
       'center_camera', 'center_camera', 'center_camera', 'center_camera'], dtype=object)

In [19]:
 class PilotNet(object):
    
    def __init__(self, sess, log_dir, n_epochs, batch_size):
        self._sess = sess
        self._log_dir = log_dir
        self._n_epochs = n_epochs
        self._batch_size = batch_size
        self._build_graph()

    def _model(self, x):
        assert(x[0].shape == (480,640,3))
        out = tf.layers.batch_normalization(x)
        out = tf.layers.conv2d(x, 24, [5,5], (2,2), "valid", activation=tf.nn.relu)
        out = tf.layers.conv2d(out, 36, [5,5], (2,2), "valid", activation=tf.nn.relu)
        out = tf.layers.conv2d(out, 48, [5,5], (2,2), "valid", activation=tf.nn.relu)
        out = tf.layers.conv2d(out, 64, [3,3], (1,1), "valid", activation=tf.nn.relu)
        out = tf.layers.conv2d(out, 64, [3,3], (1,1), "valid", activation=tf.nn.relu)
        out = tf.reshape(out, [-1, 64*53*73])
        out = tf.layers.dense(out, 100, tf.nn.relu) 
        out = tf.layers.dense(out, 50, tf.nn.relu) 
        out = tf.layers.dense(out, 10, tf.nn.relu) 
        out = tf.layers.dense(out, 1)
        return out
    
    def _build_graph(self):
        self._inputs = tf.placeholder("float", [None, 480, 640, 3])
        self._targets = tf.placeholder("float", [None, 1])
        self._global_step = tf.Variable(0, dtype=tf.int32, trainable=False, name='global_step')

        self._predict = self._model(self._inputs)
        self._loss = tf.losses.mean_squared_error(labels=self._targets, predictions=self._predict)
        self._train =  tf.train.AdamOptimizer().minimize(self._loss, global_step=self._global_step)

    def _train_admin_setup(self):
        # Writers
        self._train_writer = tf.summary.FileWriter(self._log_dir + '/train', self._sess.graph)

        # Saver
        self._saver = tf.train.Saver()

        # Summaries
        tf.summary.scalar("loss", self._loss)
        self._all_summaries = tf.summary.merge_all()
        
    def train(self, file_path):
        self._train_admin_setup()
        tf.global_variables_initializer().run()
        
        for epoch in range(self._n_epochs):
            dataset = input_fn(file_path)
            dataset = dataset.batch(self._batch_size)
            batch_generator = dataset.make_one_shot_iterator()
            epoch_loss = 0
            for step in range(10):
                img_batch, label_batch = self._sess.run(batch_generator.get_next())
                
                loss, _ = self._sess.run([self._loss, self._train], feed_dict={
                    self._inputs: img_batch['image'], 
                    self._targets: label_batch}
                )
                
                # 10 should be number of train samples / batch size
                # Need to fix to iterate over all mini-batches.
                epoch_loss += loss/10 

            # Add summary and save checkpoint after every epoch
            s = self._sess.run(self._all_summaries, feed_dict={
                self._inputs : img_batch['image'],
                self._targets: label_batch}
            )
            self._train_writer.add_summary(s, global_step=epoch)
            self._saver.save(self._sess, 'checkpoints/pilot_net', global_step=self._global_step)
            print("Epoch: {} Loss: {}".format(epoch, epoch_loss))


        # Need to closer writers
        self._train_writer.close()



In [ ]:
file_path=home+"interpolated.csv"
tf.reset_default_graph()
with tf.Session() as sess:
    model = PilotNet(sess, "pilot_net", 10 ,32)
    model.train(file_path)

Epoch: 0 Loss: 1.33559389146
Epoch: 1 Loss: 0.0316929148394
Epoch: 2 Loss: 0.0189992283471
Epoch: 3 Loss: 0.0415217437549
Epoch: 4 Loss: 0.0133060722728
